# Stock market API tools using financialdatasets.ai

The [financial datasets](https://financialdatasets.ai/) stock market API provides REST endpoints that let you get financial data for 16,000+ tickers spanning 30+ years.

This notebook creates an Agent, which has tools for getting financial data from income statements, balance sheets, and cash flow statements for a ticker.

You can ask the Agent any financial question and the agent will pull fundamental data from financial datasets to answer your question.

In [6]:
import getpass
import os

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.agent_toolkits.financial_datasets.toolkit import (
    FinancialDatasetsToolkit,
)
from langchain_community.tools.financial_datasets.balance_sheets import BalanceSheets
from langchain_community.tools.financial_datasets.cash_flow_statements import (
    CashFlowStatements,
)
from langchain_community.tools.financial_datasets.income_statements import (
    IncomeStatements,
)
from langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper
from langchain_openai import ChatOpenAI

In [ ]:
os.environ["FINANCIAL_DATASETS_API_KEY"] = getpass.getpass()

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
api_wrapper = FinancialDatasetsAPIWrapper(
    financial_datasets_api_key=os.environ["FINANCIAL_DATASETS_API_KEY"]
)
toolkit = FinancialDatasetsToolkit(api_wrapper=api_wrapper)

In [8]:
tools = toolkit.get_tools()

print("Available tools:")
for tool in tools:
    print(tool.name + "\t- " + tool.description)

Available tools:
balance_sheets	- A wrapper around financial datasets's Balance Sheets API. This tool is useful for fetching balance sheets for a given ticker.The tool fetches balance sheets for a given ticker over a given period.The period can be annual, quarterly, or trailing twelve months (ttm).The number of balance sheets to return can also be specified using the limit parameter.
cash_flow_statements	- A wrapper around financial datasets's Cash Flow Statements API. This tool is useful for fetching cash flow statements for a given ticker.The tool fetches cash flow statements for a given ticker over a given period.The period can be annual, quarterly, or trailing twelve months (ttm).The number of cash flow statements to return can also be specified using the limit parameter.
income_statements	- A wrapper around financial datasets's Income Statements API. This tool is useful for fetching income statements for a given ticker.The tool fetches income statements for a given ticker over a g

### Define system prompt

In [28]:
system_prompt = """
You are an advanced financial analysis AI assistant equipped with specialized tools 
to access and analyze financial data. Your primary function is to help users with 
financial analysis by retrieving and interpreting income statements, balance sheets, 
and cash flow statements for publicly traded companies.

You have access to the following tools from the FinancialDatasetsToolkit:

1. Balance Sheets: Retrieves balance sheet data for a given ticker symbol.
2. Income Statements: Fetches income statement data for a specified company.
3. Cash Flow Statements: Accesses cash flow statement information for a particular ticker.

Your capabilities include:

1. Retrieving financial statements for any publicly traded company using its ticker symbol.
2. Analyzing financial ratios and metrics based on the data from these statements.
3. Comparing financial performance across different time periods (e.g., year-over-year or quarter-over-quarter).
4. Identifying trends in a company's financial health and performance.
5. Providing insights on a company's liquidity, solvency, profitability, and efficiency.
6. Explaining complex financial concepts in simple terms.

When responding to queries:

1. Always specify which financial statement(s) you're using for your analysis.
2. Provide context for the numbers you're referencing (e.g., fiscal year, quarter).
3. Explain your reasoning and calculations clearly.
4. If you need more information to provide a complete answer, ask for clarification.
5. When appropriate, suggest additional analyses that might be helpful.

Remember, your goal is to provide accurate, insightful financial analysis to 
help users make informed decisions. Always maintain a professional and objective tone in your responses.
"""

### Create LLM

In [17]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

### Set user query

In [24]:
query = "What was AAPL's revenue in 2023? What about it's total debt in Q1 2024?"

### Create agent

In [26]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)


agent = create_tool_calling_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

### Ask question

In [27]:
agent_executor.invoke({"input": query})

{'input': "What was AAPL's revenue in 2023? What about it's total debt in Q1 2024?",
 'output': "### AAPL's Financial Metrics:\n\n#### Revenue in 2023:\n- **Revenue:** $383.29 billion\n\n#### Total Debt in Q1 2024:\n- **Total Debt:** $104.59 billion\n\nThese figures provide a snapshot of Apple's financial performance and financial position. The revenue figure is taken from the annual income statement for the fiscal year ending in 2023, while the total debt figure is from the quarterly balance sheet for Q1 2024."}